In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import talib as ta

In [52]:
df = pd.read_csv('../../../data/stocks/AAPL.csv')
temp = df.copy()
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


# Buying and Selling over SMA intersection Strategy

In [53]:
def buy_sell_over_SMA_intersection_strategy(df: pd.DataFrame) -> pd.DataFrame:
    df['SMA_50'] = ta.SMA(df['Close'], timeperiod=50)
    df['Signal'] = np.where(df['Close'] > df['SMA_50'], 1, 0)
    df['Entry/Exit'] = df['Signal'].diff()
    return df


In [54]:
def plot_buy_sell_over_SMA_intersection_strategy(df: pd.DataFrame) -> go.Figure:
    fig = go.Figure(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close']
        )
    )
    fig.add_trace(
        go.Scatter(
            name='50_SMA',
            x=df['Date'],
            y=df['SMA_50'],
            line=dict(color='blue', width=1),
        )
    )

    # Filter data for Buy and Sell points
    buys = df[df['Entry/Exit'] == 1]
    sells = df[df['Entry/Exit'] == -1]

    # Add Buy Markers (Green Triangles)
    fig.add_trace(
        go.Scatter(
            x=buys['Date'],
            y=buys['Low'] * 0.99,  # Offset slightly below the candle
            mode='markers',
            name='Buy Signal',
            marker=dict(
                symbol='triangle-up',
                size=12,
                color='green',
            ),
        )
    )

    # Add Sell Markers (Red Triangles)
    fig.add_trace(
        go.Scatter(
            x=sells['Date'],
            y=sells['High'] * 1.01,  # Offset slightly above the candle
            mode='markers',
            name='Sell Signal',
            marker=dict(
                symbol='triangle-down',
                size=12,
                color='red',
            ),
        )
    )

    fig.update_layout(
        title='AAPL SMA 50 Strategy 1 - Buy/Sell on SMA Intersection',
        xaxis_rangeslider_visible=False,
        yaxis_title='Price (USD)',
        template='plotly_dark' 
    )

    return fig

In [55]:
df = buy_sell_over_SMA_intersection_strategy(df)
plot_buy_sell_over_SMA_intersection_strategy_figure = plot_buy_sell_over_SMA_intersection_strategy(df)
plot_buy_sell_over_SMA_intersection_strategy_figure.show()